In [53]:
import pandas as pd

df = pd.read_csv("data/selected_data.csv")

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
df.head(5)

,Politikbereich,Zweck
0,Verkehr,"Nord-Süd-Tangente; Linie 26/27, 2.2. Teil-BPU,A"
1,Bildung,Gedenken zu 30 Jahre Mauerfall
2,Stadtentwicklung,Lernen Na Logo - Bildungsnetzwerk Hellersdorfer Promenade
3,Jugend,Kinder- und Jugendambulanz
4,Jugend,Therapiebad


In [5]:
tmp_df = df.groupby(["Politikbereich"]).count().sort_values(by="Zweck", ascending = False).reset_index()
max = tmp_df["Zweck"].max()
tmp_df["missing"] = tmp_df["Zweck"].apply(lambda n:478-n)
tmp_df.head(3)

,Politikbereich,Zweck,missing
0,Arbeit,478,0
1,Wirtschaft,177,301
2,Kultur,95,383


In [58]:
pd.DataFrame(list(zip(["a","a","a"],["b","b","b"])), columns=["Politikbereich","Zweck"])

,Politikbereich,Zweck
0,a,b
1,a,b
2,a,b


In [61]:
import nlpaug.augmenter.word as naw

aug_insert = naw.ContextualWordEmbsAug(model_path='bert-base-german-cased', action="insert")
aug_substitute = naw.ContextualWordEmbsAug(model_path='bert-base-german-cased', action="substitute")

augmented_text = []
politikbereiche = []

for Politikbereich, missing in zip(tmp_df["Politikbereich"].values, tmp_df["missing"].values):
    original_sentences = df[df["Politikbereich"]==Politikbereich]["Zweck"].values
    original_sentences_size = original_sentences.size - 1
    for i in range(missing):
        if i % 2 == 0:
            augmented_text.append(aug_insert.augment(original_sentences[round(i/missing*original_sentences_size)]))
        else:
            augmented_text.append(aug_substitute.augment(original_sentences[round(i/missing*original_sentences_size)]))
        politikbereiche.append(Politikbereich)

In [62]:
new_data = pd.DataFrame(list(zip(politikbereiche,augmented_text)), columns=["Politikbereich","Zweck"])

In [64]:
df = df.append(new_data, ignore_index=True)
df.tail(5)

C:\Users\Lisandro\AppData\Local\Temp\ipykernel_17184\3050039684.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_data, ignore_index=True)


,Politikbereich,Zweck
11467,Verbraucherschutz,IFAT Köln 2020 Spiele in Berlin München 07. 09. - 11. 03 09. Dresden 2020
11468,Verbraucherschutz,"Aufbau einer erweiterten beruflichen Perspektive zum Vergleich der Arbeiten aller akademischen Studierenden, entwickelt speziell für interessierte Studenten von TU"
11469,Verbraucherschutz,"Aufbau einer erweiterten der analytischen Methode Plattform zum Vergleich hinsichtlich der jeweiligen Leistungen der akademischen Labore, entwickelt jedoch speziell schon für die damaligen Studenten der Freien Naturwissenschaften"
11470,Verbraucherschutz,"Mit einer modernen offiziellen Plattform zum Untersuchungen der Bewertungen an akademischen Labore, und speziell für die Studenten der Universitäten"
11471,Verbraucherschutz,Nach Betriebsverlagerung


In [66]:
df.groupby(["Politikbereich"]).count().sort_values(by="Zweck", ascending = False).reset_index()

,Politikbereich,Zweck
0,Antidiskriminierung,478
1,Arbeit,478
2,Wirtschaft,478
3,Verkehr,478
4,Verbraucherschutz,478
5,Umwelt,478
6,Stadtentwicklung,478
7,Sport,478
8,Soziales,478
9,"Sicherheit, Ordnung",478


In [74]:
print(len(df))
copy = df.copy()
copy = copy.drop_duplicates()
print(len(copy))

11472
11437


Observation: Only ~40 examples are duplicated, we can remove those.

In [75]:
df.drop_duplicates().to_csv("data/augmented_data.csv", index = False)